In [3]:
import pandas as pd
import tensorflow as tf

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [3]:
data = pd.read_csv("/kaggle/input/early-detection-of-3d-printing-issues/train.csv")

In [4]:
data.head()

,img_path,printer_id,print_id,has_under_extrusion
0,101/1678589738/1678589914.060332.jpg,101,1678589738,1
1,101/1678589738/1678589914.462857.jpg,101,1678589738,1
2,101/1678589738/1678589914.875075.jpg,101,1678589738,1
3,101/1678589738/1678589915.284495.jpg,101,1678589738,1
4,101/1678589738/1678589915.700486.jpg,101,1678589738,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81060 entries, 0 to 81059
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   img_path             81060 non-null  object
 1   printer_id           81060 non-null  int64 
 2   print_id             81060 non-null  int64 
 3   has_under_extrusion  81060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 2.5+ MB


In [6]:
data["img_path"]= ["/kaggle/input/early-detection-of-3d-printing-issues/images/" + each for each in data["img_path"]]

In [7]:
data = data.sample(frac = 1)
train, val = data[0:int(0.8*len(data))], data[int(0.8*len(data)):]

In [8]:
train.head()

,img_path,printer_id,print_id,has_under_extrusion
47333,/kaggle/input/early-detection-of-3d-printing-i...,103,1679009928,1
65370,/kaggle/input/early-detection-of-3d-printing-i...,104,1678414274,0
14561,/kaggle/input/early-detection-of-3d-printing-i...,101,1678341667,0
60308,/kaggle/input/early-detection-of-3d-printing-i...,104,1679270824,1
6671,/kaggle/input/early-detection-of-3d-printing-i...,101,1678593348,1


In [9]:
val.head()

,img_path,printer_id,print_id,has_under_extrusion
50175,/kaggle/input/early-detection-of-3d-printing-i...,103,1678832038,0
60425,/kaggle/input/early-detection-of-3d-printing-i...,104,1679270824,1
43945,/kaggle/input/early-detection-of-3d-printing-i...,103,1679007451,1
54656,/kaggle/input/early-detection-of-3d-printing-i...,104,1678453862,1
46225,/kaggle/input/early-detection-of-3d-printing-i...,103,1679009928,1


In [10]:
temp = tf.io.read_file(data["img_path"][250])
temp = tf.image.decode_png(temp, channels=3)
print(temp.shape)

(480, 640, 3)


In [11]:
train_x = train["img_path"]
train_y = train["has_under_extrusion"]

val_x = val["img_path"]
val_y = val["has_under_extrusion"]

def load(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, size=(224, 224))
    return img

def one_hot(labels):
    return tf.one_hot(labels,depth=2)

train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).map(lambda xx, yy: (load(xx), one_hot(yy))).shuffle(buffer_size=1000).batch(32)
val_ds = tf.data.Dataset.from_tensor_slices((val_x, val_y)).map(lambda xx, yy: (load(xx), one_hot(yy))).shuffle(buffer_size=1000).batch(32)


In [18]:
len(train_ds)

2027

In [13]:
next(iter(train_ds))

(<tf.Tensor: shape=(32, 224, 224, 3), dtype=float32, numpy=
 array([[[[8.62184882e-01, 6.70027971e-01, 5.40616214e-01],
          [8.60534251e-01, 6.64455771e-01, 5.19357741e-01],
          [8.73949647e-01, 6.81792736e-01, 5.21008432e-01],
          ...,
          [3.76844466e-01, 2.56855965e-01, 1.56065911e-01],
          [4.13165510e-01, 2.91596860e-01, 1.81792930e-01],
          [3.74544919e-01, 2.57738173e-01, 1.63620532e-01]],
 
         [[8.79832029e-01, 6.83753550e-01, 5.54341793e-01],
          [9.01120484e-01, 7.02521026e-01, 5.57422996e-01],
          [9.11074460e-01, 7.07152843e-01, 5.50290108e-01],
          ...,
          [3.78452599e-01, 2.51391202e-01, 1.43418193e-01],
          [3.98739547e-01, 2.65406191e-01, 1.62044823e-01],
          [3.19931984e-01, 2.08867550e-01, 1.30296126e-01]],
 
         [[8.79551947e-01, 6.83473468e-01, 5.49860001e-01],
          [9.10084069e-01, 7.06162453e-01, 5.61064422e-01],
          [9.21788752e-01, 7.10364163e-01, 5.59353769e-01],
    

In [12]:
#resnet
model = tf.keras.applications.resnet_v2.ResNet101V2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None,
    classes=1000,
    classifier_activation="sigmoid"
)


inputs = tf.keras.Input(shape=(224, 224, 3))
x = model(inputs, training=True)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(2,activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

171317808/171317808 [==============================] - 1s 0us/step


In [14]:
model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None,
    classes=1000,
    classifier_activation="sigmoid"
)

inputs = tf.keras.Input(shape=(224, 224, 3))
x = model(inputs, training=True)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)


219055592/219055592 [==============================] - 1s 0us/step


In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 38400)             0         
                                                                 
 dense (Dense)               (None, 2)                 76802     
                                                                 
Total params: 54,413,538
Trainable params: 54,352,994
Non-trainable params: 60,544
_________________________________________________________________


In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer=optimizer,loss=tf.keras.losses.binary_crossentropy, metrics = ["accuracy"])

In [ ]:
history = model.fit(train_ds,epochs = 50,batch_size=32,validation_data = val_ds)

Epoch 1/50
2027/2027 [==============================] - 1340s 658ms/step - loss: 0.6682 - accuracy: 0.9186 - val_loss: 1.0932 - val_accuracy: 0.9053
Epoch 2/50
2027/2027 [==============================] - 1331s 653ms/step - loss: 0.4215 - accuracy: 0.9330 - val_loss: 0.1518 - val_accuracy: 0.9532
Epoch 3/50
2027/2027 [==============================] - 1334s 655ms/step - loss: 0.4349 - accuracy: 0.9232 - val_loss: 0.2882 - val_accuracy: 0.9358
Epoch 4/50
2027/2027 [==============================] - 1344s 659ms/step - loss: 0.3520 - accuracy: 0.9262 - val_loss: 0.4015 - val_accuracy: 0.9093
Epoch 5/50
2027/2027 [==============================] - 1336s 656ms/step - loss: 0.3768 - accuracy: 0.8896 - val_loss: 0.3113 - val_accuracy: 0.9104
Epoch 6/50
2027/2027 [==============================] - 1345s 660ms/step - loss: 0.3543 - accuracy: 0.8934 - val_loss: 0.2324 - val_accuracy: 0.9258
Epoch 7/50
2027/2027 [==============================] - 1352s 663ms/step - loss: 0.2543 - accuracy: 0.9126

In [1]:
python

NameError: name 'python' is not defined